### Trabalho Prático 2 – Soluções para problemas difíceis

#### Importações

In [9]:
import numpy as np
from ucimlrepo import fetch_ucirepo
import time
import warnings
from itertools import cycle, islice
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score
import os

In [2]:
!pip install ucimlrepo

 #### Distância de Minkowski

In [16]:
# p = 1 (Distância Manhattan)
# p = 2 (Distância Euclidiana)

def minkowski(x, y, p):
   return np.power(np.sum(np.abs(x - y) ** p), 1/p)

In [15]:
def matriz_dist(df, p):
    npontos = df.shape[0]
    distancias = np.zeros((npontos, npontos))

    for i in range(npontos):
        for j in range(npontos):
            ponto1 = df.iloc[i].values
            ponto2 = df.iloc[j].values
            distancias[i, j] = minkowski(ponto1, ponto2, p)

    return distancias

#### Algoritmos aproximados K-Centros

##### Algoritmo baseado em refinamento de intervalos

In [37]:
def rvalid(npontos, raio, distancias, k):
  s_aux = np.arange(npontos)
  C = []
  while len(s_aux)>0:
    C.append(s_aux[0])
    s_aux = s_aux[1:]
    s_aux = [x for x in s_aux if distancias[C[-1], x] > 2*raio]
  return len(C) <= k, C

In [22]:
def calculo_rmax(distancias, npontos):
  rmax = -1
  for i in range(npontos):
    for j in range(npontos):
      rmax = max(rmax, distancias[i, j])

  return rmax

In [33]:
def kcentros1(npontos, distancias,k, ref):
  rmax = calculo_rmax(distancias, npontos)
  l = 0;
  r = rmax;

  while((r-l)/rmax > ref):
    m = (l+r)/2.0
    v, C = rvalid(npontos, m, distancias, k)
    if(v):
      r = m
    else:
      l = m+1
  while (l != r):
    m = (l+r)/2
    v, C = rvalid(npontos, m, distancias, k)
    if(v):
      return C
    else:
      l = m+1
  v, C = rvalid(npontos, l, distancias, k)
  return C

Os intervalos considerados serão de 5%, 10%, 15%, 20% e 25%

#### Algoritmo baseado na maximização da distância entre os centros previamente escolhidos.

In [29]:
def kcentros2(k, npontos, distancias):
  if(k > npontos):
    return np.arange()
  C = [0]
  idponto = 0
  while(len(C) < k):
    dist = -1
    for c in C:
      for i in range(npontos):
        if(distancias[i,c] > dist):
          idponto = i
          dist = distancias[i, c]
    C.append(idponto)
  return C

#### Define o indice do cluster para cada um dos dados

In [19]:
def cluster_dados(pontos, centros, distancias):
  labels = []
  for i in range(pontos.shape[0]):
    dist_centro = float('inf')
    id_cluster = 0
    for c in centros:
      if(distancias[i, c] < dist_centro):
        dist_centro = distancias[i, c]
        id_cluster = c
    labels.append(id_cluster)

  return labels

#### Cálculo do raio da solução

In [18]:
def raio(labels, distancias):
  raio = -1
  for i in range(len(labels)):
    raio = max(raio, distancias[i,labels[i]])

  return raio

#### Experimentos com o kcentros1

In [45]:
def experimentos_kcentros1(m_dist, n_pontos,  X, y_map ,t_dist, id_dataset, k):
  ref = [0.05, 0.1, 0.15, 0.2, 0.25]
  for i in range(5):
    tmp_exec_kmeans = []
    silhueta_kmeans = []
    indice_rand_kmeans = []
    raio_exec_kmeans = []

    for j in range(30):
      start_time = time.time()

      C = kcentros1(n_pontos, m_dist, k, ref[i])

      end_time = time.time()
      execution_time = end_time - start_time

      c_dados = cluster_dados(X, C, m_dist)

      tmp_exec_kmeans.append(execution_time)


      if len(np.unique(c_dados)) > 1:
        silhueta_kmeans.append(silhouette_score(X, c_dados))
      else:
        silhueta_kmeans.append(-1)

      indice_rand_kmeans.append(adjusted_rand_score(y_map, c_dados))

      raio_exec_kmeans.append(raio(c_dados, m_dist))

    # Retornar as médias e desvios-padrão das métricas
    data = {
        'Algoritmo': 'K-Centros 1',
        'Tempo Execução (s)': f"{np.mean(tmp_exec_kmeans):.4f} ± {np.std(tmp_exec_kmeans):.4f}",
        'Silhueta': f"{np.mean(silhueta_kmeans):.4f} ± {np.std(silhueta_kmeans):.4f}",
        'Índice de Rand Ajustado': f"{np.mean(indice_rand_kmeans):.4f} ± {np.std(indice_rand_kmeans):.4f}",
        'Raio Máximo': f"{np.mean(raio_exec_kmeans):.4f} ± {np.std(raio_exec_kmeans):.4f}"
    }

    df = pd.DataFrame(data, index=[0])
    df.to_csv(f'dataset_exec_kcentros1_{t_dist}_{[ref[i]]}_dataset_{id_dataset}.csv', index=False)

#### Experimentos com o kcentros2

In [26]:
def experimentos_kcentros2(m_dist, npontos, X, y_map, t_dist, id_dataset, k):
    tmp_exec_kmeans = []
    silhueta_kmeans = []
    indice_rand_kmeans = []
    raio_exec_kmeans = []

    for i in range(30):
      start_time = time.time()

      C = kcentros2(k, npontos, m_dist)

      end_time = time.time()
      execution_time = end_time - start_time

      c_dados = cluster_dados(X, C, m_dist)

      tmp_exec_kmeans.append(execution_time)
      silhueta_kmeans.append(silhouette_score(X, c_dados))
      indice_rand_kmeans.append(adjusted_rand_score(y_map, c_dados))

      raio_exec_kmeans.append(raio(c_dados, m_dist))

    data = {
        'Algoritmo': 'K-Centros 2',
        'Tempo Execução (s)': f"{np.mean(tmp_exec_kmeans):.4f} ± {np.std(tmp_exec_kmeans):.4f}",
        'Silhueta': f"{np.mean(silhueta_kmeans):.4f} ± {np.std(silhueta_kmeans):.4f}",
        'Índice de Rand Ajustado': f"{np.mean(indice_rand_kmeans):.4f} ± {np.std(indice_rand_kmeans):.4f}",
        'Raio Máximo': f"{np.mean(raio_exec_kmeans):.4f} ± {np.std(raio_exec_kmeans):.4f}"
    }

    df = pd.DataFrame(data, index=[0])
    df.to_csv(f'dataset_exec_kcentros2_{t_dist}_dataset_{id_dataset}.csv', index=False)

In [27]:
def carregar_datasets(path, num_datasets, sintetico):
    datasets = []
    for i in range(num_datasets):
        if(sintetico ==1):
          filename = os.path.join(path, f'dataset_{i}.csv')
        elif(sintetico ==2):
          filename = os.path.join(path, f'dataset_{i}_dist_mult.csv')

        df = pd.read_csv(filename)
        datasets.append(df)
    return datasets

def processar_datasets(datasets, sintetico):
    resultados = []
    for i, df in enumerate(datasets):

      if(sintetico ==1):
        X = df[['feature_1', 'feature_2']]
        y = df['target']

      elif(sintetico ==2):
        X = df[['X1', 'X2']]
        y = df['Center']

      k = y.nunique()

      distancia_euclidiana =  matriz_dist(X,2)
      distancia_manhattan =  matriz_dist(X,1)

      experimentos_kcentros2(distancia_euclidiana, X.shape[0], X, y, 'Euclidiana', i, k)
      experimentos_kcentros1(distancia_manhattan, X.shape[0], X, y, 'Manhattan', i, k)



In [46]:
##Sinteticos 1
datasets = carregar_datasets('.', 10, 1)
processar_datasets(datasets, 1)

##Sinteticos 2
datasets = carregar_datasets('.', 10, 2)
processar_datasets(datasets, 2)


#### Avaliação algoritmos aproximados

##### Dados sintéticos

**Abordagem do scikit-learn**

In [ ]:
import time
import warnings
from itertools import cycle, islice

import matplotlib.pyplot as plt
import numpy as np

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler

# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 700
seed = 30
noisy_circles = datasets.make_circles(
    n_samples=n_samples, factor=0.5, noise=0.05, random_state=seed
)
circles = datasets.make_circles(
    n_samples=n_samples, factor=0.9, random_state=seed
)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=0.05, random_state=seed)
noisy_moons_2 = datasets.make_moons(n_samples=n_samples, noise=0.09, random_state=seed)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=seed)
blobs_2 = datasets.make_blobs(n_samples=n_samples, random_state=40)
rng = np.random.RandomState(seed)
no_structure = rng.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(
    n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state
)

varied_2 = datasets.make_blobs(
    n_samples=n_samples, cluster_std=[2.0, 1.4, 0.7], random_state=random_state
)

gaussian_quantile = datasets.make_gaussian_quantiles(random_state=random_state)

# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(9 * 2 + 3, 13))
plt.subplots_adjust(
    left=0.02, right=0.98, bottom=0.001, top=0.95, wspace=0.05, hspace=0.01
)

plot_num = 1

default_base = {
    "quantile": 0.3,
    "eps": 0.3,
    "damping": 0.9,
    "preference": -200,
    "n_neighbors": 3,
    "n_clusters": 3,
    "min_samples": 7,
    "xi": 0.05,
    "min_cluster_size": 0.1,
    "allow_single_cluster": True,
    "hdbscan_min_cluster_size": 15,
    "hdbscan_min_samples": 3,
    "random_state": 42,
}

datasets = [noisy_circles, circles, noisy_moons, noisy_moons_2, varied, varied_2, aniso, blobs, blobs_2, gaussian_quantile]

for i_dataset, dataset in enumerate(datasets):
    X, y = dataset
    df = pd.DataFrame(X, columns=['feature_1', 'feature_2'])
    if y is not None:
        df['target'] = y
    df.to_csv(f'dataset_{i_dataset}.csv', index=False)


<Figure size 2100x1300 with 0 Axes>

**Distribuição normal multivariada**

In [ ]:
# lista aleatória com as posições dos centros

def generate_centers(num_centers, min_size, max_size, num_dimensions):
    centers = []
    for _ in range(num_centers):
        size = np.random.randint(min_size, max_size + 1)
        center = np.random.uniform(-10, 10, size=(size, num_dimensions))
        centers.append(center)
    return centers

num_centers = 10
min_size = 2
max_size = 9
num_dimensions = 2

centers = generate_centers(num_centers, min_size, max_size, num_dimensions)

In [ ]:
def generate_multivariate_data(centers, std_dev, num_samples_per_center):
    data = []
    for i, center in enumerate(centers):
        samples = np.random.multivariate_normal(center, np.diag([std_dev, std_dev]), num_samples_per_center)
        df_samples = pd.DataFrame(samples, columns=['X1', 'X2'])
        df_samples['Center'] = i  # Adiciona um rótulo para identificar o centro
        data.append(df_samples)

    return pd.concat(data, ignore_index=True)

import random

std_dev_min = 0.5
std_dev_max = 5.0
std_dev_values = np.linspace(std_dev_min, std_dev_max, 10)
num_samples_per_center = [random.randint(100, 200) for _ in range(10)]

for i in range(10):
  df = generate_multivariate_data(centers[i], std_dev_values[i], num_samples_per_center[i])
  df.to_csv(f'dataset_{i}_dist_mult.csv', index=False)

In [51]:
import pandas as pd
import os

# Lista para armazenar todos os DataFrames
dataframes = []

# Percorre todos os arquivos no diretório atual
for file in os.listdir():
    # Verifica se o arquivo termina com .csv
    if file.endswith(".csv"):
        # Lê o CSV em um DataFrame e adiciona à lista
        df = pd.read_csv(file)
        dataframes.append(df)

# Concatena todos os DataFrames em um único DataFrame
df_consolidado = pd.concat(dataframes, ignore_index=True)

# Salva o DataFrame consolidado em um novo arquivo CSV
df_consolidado.to_csv('tabela_consolidada.csv', index=False)

# Baixa o arquivo CSV consolidado
from google.colab import files
files.download('tabela_consolidada.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
import pandas as pd
import os
import re

# Função para processar os arquivos CSV
def processar_arquivos_csv():
    for file in os.listdir():
        # Verifica se o arquivo termina com .csv
        if file.endswith(".csv"):
            # Procura o padrão dentro dos colchetes no nome do arquivo
            match = re.search(r'\[(.*?)\]', file)
            if match:
                # Extrai o valor dentro dos colchetes
                valor = match.group(1)
                # Lê o arquivo CSV
                df = pd.read_csv(file)
                # Modifica o conteúdo da primeira coluna, adicionando o valor extraído
                df.iloc[:, 0] = df.iloc[:, 0].astype(str) + "_" + valor
                # Salva o arquivo modificado
                df.to_csv(file, index=False)
            else:
                # Se não houver colchetes, apenas lê e salva o arquivo sem modificações
                df = pd.read_csv(file)
                df.to_csv(file, index=False)

# Executa a função para processar os arquivos CSV
processar_arquivos_csv()
